# IMPORTO LE LIBRERIE PRINCIPALI

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


# MI LEGGO I VARI FILE CHE SONO DI SUPPORTO

In [ ]:
train=pd.read_csv('train.csv')

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
pokemon=pd.read_csv('pokemon.csv')

# DO UNA RAPIDA OCCHIATA AL DATASET

In [ ]:
train.head()


In [ ]:
test.head()

# CAMBIO I VALORI DI WINNER, SE VINCE IL PRIMO VALE 1 ALTRIMENTI SE VINCE IL SECONDO VALE 0

In [ ]:
def binary_winner(winner, first_pokemon, second_pokemon):
    if (first_pokemon == winner):
        return 1
    elif (second_pokemon == winner):
        return 0


In [ ]:
train['Winner']= train.apply(lambda x: binary_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)
test['Winner']= test.apply(lambda x: binary_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)

# INSERISCO TRAIN E TEST IN UN UNICO DATASET CHIAMO DATASET IN CUI HO UNA COLONNA CHIAMATA TRAIN IL CUI VALORE VALE 1 SE IL RECORD E' PRESENTE NEL TRAIN 0 SE IL RECORD E' PRESENTE NEL TEST

In [ ]:
train["train"]=1
test["train"]=0
dataset=pd.concat([train, test], ignore_index=True)

In [ ]:
dataset['train']

In [ ]:
list(train)

In [ ]:
list(test)

In [ ]:
train.info()

In [ ]:
pokemon.info()

# ESEGUO UN MERGE FRA IL DATASET "POKEMON" E IL DATASET "DATASET"

In [ ]:
joined_data= dataset.join(pokemon.set_index('#'), on ='First_pokemon', rsuffix="_first")

In [ ]:
joined_data= joined_data.join(pokemon.set_index('#'), on ='Second_pokemon', rsuffix="_second")

# VEDO LE FEATURE COMPLETE

In [ ]:
list(joined_data)

### MI STAMPO LA QUANTITA' DEI VALORI CHE ASSUMONO LE VARIE FEATURES

In [ ]:
for c in joined_data.columns:
    print ("Features: ", c)
    print(joined_data[c].value_counts())
    print("\n*****************\n")

# VERIFICO SE CI SONO DEI VALORI MANCANTI ALL'INTERNO DEL DATASET

In [ ]:
joined_data.info()

# VALORI NULLI
- Name
- Name_second
- Type 2 
- Type 2_second

In [ ]:
print(joined_data.isnull().sum())

# Type2 ha molti valori mancanti!
quasi la metà, potrei pensare di dropparla, ma dato che il tipo è una feature che dovrebbe essere rilevante nei Pokemon ai fini della sfida, portei pensare di mettere al posto del valore mancante di Type2 il valore di Type1 

In [ ]:
joined_data['Type 2'].fillna(joined_data['Type 1'],inplace=True)
joined_data['Type 2_second'].fillna(joined_data['Type 1_second'],inplace=True)

# "NAME"  DATO CHE E' UNICA SI PUO' DROPPARE

In [ ]:
print(joined_data.isnull().sum())

In [ ]:
len(pokemon['Name'].unique())

In [ ]:
toDrop = ['Name', 'Name_second']
joined_data = joined_data.drop(toDrop, axis=1)

In [ ]:
joined_data.head()

##  Encoding: i modelli ragionano in termini di NUMERI e non di stringhe. 
## Per questo motivo, bisogna convertire i campi object/bool in numeri.

In [ ]:
# modifico alcuni nomi delle feature per comodità

In [ ]:
joined_data.rename(columns={'Type 2':'Type2'},inplace=True)
joined_data.rename(columns={'Type 1':'Type1'},inplace=True)

In [ ]:
joined_data.rename(columns={'Type 1_second':'Type1_second'},inplace=True)
joined_data.rename(columns={'Type 2_second':'Type2_second'},inplace=True)
joined_data.rename(columns={'Sp. Atk':'Sp_Atk'},inplace=True)
joined_data.rename(columns={'Sp. Def':'Sp_Def'},inplace=True)
joined_data.rename(columns={'Sp. Atk_second':'Sp_Atk_second'},inplace=True)
joined_data.rename(columns={'Sp. Def_second':'Sp_Def_second'},inplace=True)

In [ ]:
joined_data.dtypes

## i tipi da convertire sono:
- Type1
- Type2
- Type2_second
- Type1_second
- Legendary
- Legendary_second


In [ ]:
from sklearn.preprocessing import LabelEncoder
Feature = ["Type1", "Type2","Type2_second", "Legendary", "Type1_second", "Legendary_second"]
labelEncoder_X = LabelEncoder()
for element in Feature:
    joined_data[element] = labelEncoder_X.fit_transform(joined_data[element])
    
joined_data.head()

In [ ]:
list(joined_data)

In [ ]:
joined_data.head()

In [ ]:
train= joined_data[joined_data.train == 1].drop("train", axis=1)
test= joined_data[joined_data.train == 0].drop("train", axis=1)



In [ ]:
y_train = train['Winner']           # vector of labels (dependent variable) #cosa dobbiamo predirre
X_train = train.drop(['Winner'], axis=1)      # remove the dependent variable from the dataframe Train #tutto tranne la label

y_test = test['Winner']           # vector of labels (dependent variable) #cosa dobbiamo predirre
X_test = test.drop(['Winner'], axis=1)      # remove the dependent variable from the dataframe Test #tutto tranne la label


## Mi stampo un grafico che mostra quanti come sono distribuiti i valori per Winner nel dataset 

In [ ]:
sns.countplot(x=train['Winner'])

In [ ]:
sns.countplot(x=test['Winner'])

Notiamo che i valori sono distribuiti in maniera abbastanza omogenea sia nel dataset di train sia nel dataset di test

In [ ]:
sns.countplot(x=train["Generation"],hue=train["Winner"]);

La feature Generation sembrerebbe superflua.

In [ ]:
plt.figure(figsize=(30,30))
sns.countplot(x=train["Speed"],hue=train["Winner"]);

Dal grafico di sopra possiamo notare come 
### i pokemon che hanno velocità più alta tendono a vincere. 

Definisco quindi una nuova Feature che mi rappresenta la differenza fra le Stats Velocità dei due Pokemon

In [ ]:
train['diff_Speed']=train['Speed']-train['Speed_second']
test['diff_Speed']=test['Speed']-test['Speed_second']

In [ ]:
plt.figure(figsize=(80,100))
sns.countplot(x=train["diff_Speed"],hue=train["Winner"]);

### DA QUESTO GRAFICO RISULTA MOLTO PIU' EVIDENTE CIO' CHE HO AFFERMATO PRECEDENTEMENTE
OVVERO CHE I POKEMON CHE POSSIEDONO VELOCITA' PIU ALTE TENDONO A VINCERE. 
- INFATTI DIFFERENZA POSITIVA VUOL DIRE CHE IL PRIMO E' PIU' VELOCE DEL SECONDO -> VINCE IL PRIMO, E INFATTI WINNER=1
- INVECE DIFFERENZA NEGATIVA VUOL DIRE CHE IL SECONDO E' PIU' VELOCE DEL PRIMO -> VINCE IL SECONDO, E INFATTI WINNER=0

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["HP"],hue=train["Winner"]);

DA QUESTO GRAFICO SEMBREREBBE CHE HP POSSA ESSERE UTILE, PER CAPIRE MEGLIO L'ANDAMENTO DEFINISCO ANCHE PER QUESTO UNA NUOVA FEATURE 

In [ ]:
train['diff_HP']=train['HP']-train['HP_second']
test['diff_HP']= test['HP']- test['HP_second']

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["diff_HP"],hue=train["Winner"]);

DA QUESTO GRAFICO SEMBREREBBE CHE HP POSSA ESSERE UTILE, anche se risulta che con differenza positiva bassa continua a vincere il secondo 

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["Attack"],hue=train["Winner"]);

QUESTO GRAFICO COSI' NON SEMBRA ESSERE MOLTO RILEVANTE QUINDI PROVIAMO A VEDERE SE DEFINENDO UNA FEATURE CHE MI RAPPRESENTA LA DIFFERENZA FRA I DUE ATTACCHI OTTENGO UN QUALCOSA DI PIU' INTERESSANTE

In [ ]:
train['diff_Attack']=train['Attack']-train['Attack_second']
test['diff_Attack']=test['Attack']-test['Attack_second']


In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["diff_Attack"],hue=train["Winner"]);

DIFFERENZA NEGATIVA TENDEONO A VINCERE MAGGIORMENTE IL POKEMON 2 DIFFERENZA POSITIVA VICEVERSA. 
QUESTA FEATURE RISULTA ESSERE ABBASTANZA VALIDA. 
ATTACCO 1 MAGGIORE ATTACCO2--> vince 1
ATTACCO 2 MAGGIORE ATTACCO1--> vince2

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["Defense"],hue=train["Winner"]);

Anche qui con differenza negativa tende a vincere il secondo mentre con differenza positiva tende a vincere il primo. 
Non si vede in maniera eclatante come si era visto per la feature different speed

In [ ]:
train['diff_Defense']=train['Defense']-train['Defense_second']
test['diff_Defense']=test['Defense']-test['Defense_second']

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["diff_Defense"],hue=train["Winner"]);

DA QUESTO GRAFICO NON EMERGONO COSE RILEVANTI I VALORI DI VINCITA DEL PRIMO E DEL SECONDO SONO STATISTICAMENTE PARAGONABILI QUINDI PROBABILMENTE QUESTA FEATURE PUO' ESSERE DROPPATA

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["Sp_Atk"],hue=train["Winner"]);

In [ ]:

train['diff_Sp_Atk']=train['Sp_Atk']-train['Sp_Atk_second']
test['diff_Sp_Atk']=test['Sp_Atk']-test['Sp_Atk_second']

In [ ]:
sns.countplot(x=train["diff_Sp_Atk"],hue=train["Winner"]);

In [ ]:
sns.countplot(x=train["Sp_Def"],hue=train["Winner"]);

In [ ]:
train['diff_Sp_Def']=train['Sp_Def']-train['Sp_Def_second']
test['diff_Sp_Def']=test['Sp_Def']-test['Sp_Def_second']

In [ ]:
sns.countplot(x=train["diff_Sp_Def"],hue=train["Winner"]);

In [ ]:
sns.countplot(x=train["Legendary"],hue=train["Winner"]);

La feature Legendary invece ci mostra che i legendari sono pochi e quelli che ci sono se attaccano per primi tendono a vincere, mentre per i non legendari sembrerebbe che siano molti.

In [ ]:
sns.countplot(x=train["Legendary_second"],hue=train["Winner"]);

# Mi stampo una heatmap che mi guarda quale sia la correlazione che ho fra le feature:
- valore negativo -> relazione di proporzionalità inversa
- valore positivo -> relazione di proporzionalità diretta
- valore pari a zero-> no correlazione (feature può essere eliminata)

In [ ]:
plt.figure(figsize=(18,10))
correlation = train.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

Dalla heatmap vediamo che le seguenti feature non sono utili per il nostro scopo:
- First_Pokemon
- Second_Pokemon
- Defense 
- diff_Sp_Def --> come avevamo visto dai countplot 
- Generation
- Defense_second
- Generation_second
- Type1 
- Type2


Quindi si possono droppare

In [ ]:
toDrop = ['First_pokemon','Second_pokemon', 'Defense','diff_Sp_Def' ,'Generation', 'Defense_second', 'Generation_second']
train = train.drop(toDrop, axis=1)

In [ ]:
toDrop = ['First_pokemon','Second_pokemon',  'Defense','diff_Sp_Def' ,'Generation',  'Defense_second', 'Generation_second']
test = test.drop(toDrop, axis=1)

In [ ]:
decisionTree = DecisionTreeClassifier()
logisticRegression = LogisticRegression(random_state=0, max_iter = 10000)
randomForest = RandomForestClassifier()

models = [decisionTree, logisticRegression, randomForest]
score = []

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = round(accuracy_score(y_pred, y_test) * 100, 2)
    score.append(acc)
    print(model)
    print(acc)
    plot_confusion_matrix(model, X_test, y_test, values_format='n', display_labels=['No','Yes']) 
    plt.show() #sx predizioni giuste, dx predizioni sbagliate
    print(classification_report(y_test,y_pred))

In [ ]:
models = pd.DataFrame({

    'Model': ['Decision Tree','Logistic Regression', 'Random Forest'],

    'Score': score})
    
models.sort_values(by='Score', ascending=False)